In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal

from IPython.display import YouTubeVideo, HTML, Audio
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, show, output_notebook

output_notebook()

# Efectos del muestreo y enventanado en la DFT

## ¿Qué le ocurre al espectro de una señal continua cuando la muestreamos?

Muestrear es equivalente a multiplicar una señal continua por un tren de impulsos (peineta de Dirac)

$$
\text{III}_{F_s}(t) = \sum_{m=-\infty}^\infty \delta[t - m / F_s],
$$

donde $F_s$ es la frecuencia de muestreo. La transformada de Fourier del tren de impulsos es

$$
\mathbb{FT}[\text{III}_{F_s}] =  2\pi F_s \sum_{m=-\infty}^\infty  \delta(\omega - 2\pi m F_s)
$$

Multiplicar en el tiempo corresponde a convolucionar en frecuencia

$$
\begin{align}
\mathbb{FT}[s(t) \text{III}_{F_s}(t)] &= \mathbb{FT}[s(t)] * \mathbb{FT}[\text{III}_{F_s}(t)] \nonumber \\
&= S(\omega) * 2\pi F_s \sum_{m=-\infty}^\infty  \delta(\omega - 2\pi m F_s) \nonumber \\
&= 2\pi F_s \sum_{m = -\infty}^{\infty} S(\omega - 2 \pi m F_s) \nonumber
\end{align}
$$


<div class="alert alert-info">
    
**Por lo tanto:** Muestrear en el tiempo hace que el espectro de la señal continua $S(\omega)$ se repita infinitamente. El espectro se repite cada $2 \pi F_s$ [rad/s]
    
</div>

La siguiente figura expresa lo anterior

<img src="../images/fft-sampling1.png">

La siguiente figura muestra lo que ocurre si el espectro de la señal continua tiene frecuencias mayores a $\frac{F_s}{2}$

<img src="../images/fft-sampling2.png">

<div class="alert alert-info">
    
Este efecto de solapamiento entre los espectros se llama **aliasing**
    
</div>

## Aliasing

- El espectro de una señal muestreada es periódico en $F_s$
- Si originalmente la señal tenía componentes con frecuencias mayores a $\frac{F_s}{2}$ se produce un "traslape espectral"
- Este fenomeno se llama *aliasing* y los componentes traslapados se denominan *aliases*

Cuando existe aliasing no se puede reconstruir la señal original

## Teorema del muestreo


Sea una señal continua $s(t)$ muestreada a $F_s$ produciendo una señal digital $s[n] = s(n/F_s)$

Si $F_s > 2 f_{max}$, donde $f_{max}$ es la componente de frecuencia más alta de la señal, entonces la señal analógica puede ser recuperada a partir de sus muestras discretas y el valor de la señal reconstruida es:

$$
s(t) = \sum_{n=-\infty}^{\infty} s_n \text{sinc}(\pi F_s (t - n /F_s) )
$$



## ¿Cómo eliminamos el aliasing?


Debemos forzar que todas las frecuencias del espectro o al menos las de mayor importancia sean menores que $\frac{Fs}{2}$

Podemos lograr esto mediante
- Filtrado: Eliminar las frecuencias mayores a $\frac{F_s}{2}$ (Próxima unidad)
- Aumentar $F_s$ tal que sea dos veces mayor que la frecuencia máxima de interés


### Ejemplo

- Se tiene una señal con tres componentes a 1Hz, 12Hz y 23Hz, respectivamente
- Modifica la frecuencia de muestreo y detecta los componentes erroneos o aliases en el espectro
- Verifica como afectan a la reconstrucción de la señal

In [ ]:
fig, ax = plt.subplots(4, figsize=(6, 6));
t = np.arange(-3, 3, step=0.01); N = len(t)
s =  np.cos(2*np.pi*t) - 0.5*np.sin(2.0*np.pi*12*t + np.pi/3) + 0.25*np.sin(2.0*np.pi*23*t) 
ax[0].plot(t, s, '-'); ax[0].set_title('Original')

def update(Fs):    
    t_short = t[::int(100/Fs)]; s_short = s[::int(100/Fs)]
    ax[1].cla(); ax[1].plot(t_short, s_short, '.'); ax[1].set_title('Sampled at %0.2f' %(Fs))
    S = fftpack.fft(s_short); f = fftpack.fftshift(fftpack.fftfreq(n=len(S), d=1.0/Fs)); 
    ax[2].cla(); ax[2].plot(f, fftpack.fftshift((np.absolute(S))), linewidth=2);
    S_pad = np.concatenate((S[:int(len(S)/2)], np.zeros(shape=(N-len(S))), S[int(len(S)/2):]))
    ax[2].set_title('Spectrum '); s_recon = np.real(fftpack.ifft(S_pad))
    ax[3].cla(); ax[3].plot(t, s_recon*len(t)/len(t_short), '-', linewidth=2); 
    ax[3].set_title("Reconstruction from padded spectrum");
interact(update, Fs=SelectionSlider_nice(options=[5, 10, 25, 33, 50, 100], value=100));

cosas viejas

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(7, 4))
t = np.linspace(-10, 10, num=1000); s = np.sinc(t)
line = ax.plot(t, s)
def update(n = 1): 
    line[0].set_ydata(np.sinc(0.01*n*t))
    ax.set_title("Frequency: %0.2f [Hz]" %(0.01*n/2))
anim = animation.FuncAnimation(fig, update, frames=200, interval=100, blit=True) 

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(7, 5))
t = np.arange(0, 10.01, step=1/640.) # 1[s] = 640 samples
s = harmonic(t, f=1.5124) # fmax = 1 Hz
def update(dt, ds):
    Nts = int(640*dt)
    fs, sn = (640.)/Nts, quantize(s[::Nts], ds)
    for axis in ax:
        axis.cla()
        format_plot(axis, Nts, ds)    
    ax[0].plot(t[::Nts], sn, linestyle='--', marker='o', markersize=12)
    ax[0].set_title("Sampling frequency: %0.2f" %(fs)); s_recon = 0.0
    ax[1].plot(t, s, linewidth=2, label='original');
    for k in range(len(sn)):
        s_recon += sn[k]*np.sinc(fs*(t - k/fs))
    ax[1].plot(t, s_recon, linewidth=2, linestyle='--', label='reconstructed')
    plt.legend(loc=1);
interact(update, 
         dt=SelectionSlider_nice(description="$T_s$", 
                                 options=[1./32, 1./16, 1./8, 1./4, 1/3, 1./2], value=0.25), 
         ds=SelectionSlider_nice(description="$\Delta s$", 
                                 options=[0.125, 0.25, 0.5, 1.0, 2.0], value=0.25));

## Principio o Teorema de incertidumbre


El principio de incertidumbre de Heisenberg nos dice que la precisión (certeza) con que medimos la posición de una particula es inversamente proporcional a la precisión con que medimos su momentum lineal:

$$
\Delta x \Delta p \geq \frac{h}{4\pi},
$$

donde $h$ es la constante de Planck. 

**En señales existe un principio análogo:** No podemos especificar con infinita precisión la localización temporal y frecuencial de una señal al mismo tiempo.

***

### Ejemplo: Pulso cuadrado

Teníamos

$$
s(t) = \begin{cases} 1, & |t| < T \\ 0, & |t| > T\end{cases}
$$

con

$$
S(\omega) = \mathbb{FT}[s(t)] = 2T \text{sinc}(\omega T)
$$

y notamos que a mayor ancho en tiempo menor ancho en frecuencia y viceversa. 

Definiendo la resolución o ancho de la señal como la distancia entre sus cruces por cero, es fácil verificar que:

- $\Delta t = 2T$
- $\Delta \omega =  \omega|_{\text{sinc}(- \omega T) = 0}^{\text{sinc}(\omega T) = 0} = \frac{2\pi}{T}$
- $\Delta t \Delta \omega = 4 \pi$

Es decir a menor ancho en el tiempo, mayor ancho en frecuencia y viceverza

In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, 2, figsize=(6, 3))
t = np.linspace(-5, 5, num=500); f = np.linspace(-5, 5, num=500)
ax[0].set_xlabel('Time [s]'); ax[1].set_xlabel('Frequency [Hz]');
line_square = ax[0].plot(t, np.zeros_like(t), linewidth=4); ax[0].set_ylim([-.1, 1.1])
line_gabor1 = ax[0].plot([-1, 1], [0, 0], 'r--')
line_sinc = ax[1].plot(f, np.zeros_like(f), linewidth=4)
line_gabor2 = ax[1].plot([-0.5, 0.5], [0, 0], 'r--')

def update(T):    
    s = np.zeros_like(t); s[(t> - T) & (t<  T)] = 1    
    line_square[0].set_ydata(s);  line_gabor1[0].set_xdata([-T, T])
    S = 2*T*np.sinc(2*f*T); line_sinc[0].set_ydata(S); 
    line_gabor2[0].set_xdata([-0.5/T, 0.5/T])
    ax[1].set_ylim([-2*T/np.pi, 2.2*T])
    
interact(update, T=SelectionSlider_nice(options=[1/8, 1/4, 1/2, 1, 2, 4], value=1));

***

### Intuición: Resolución frecuencial de una sinusoide

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(7, 3))
t = np.arange(-4, 4, step=1e-2)
def update(dt):
    ax.cla(); ax.plot(t, np.cos(2.0*np.pi*t))
    ax.plot(t, np.cos(2.0*np.pi*(1.0+0.125/dt)*t))
    rect = patches.Rectangle((-dt, -1), 2*dt, 2, fill=False, lw=2)
    ax.add_patch(rect)
interact(update, dt=SelectionSlider_nice(options=[0.5, 1., 2.]));

Denis Gabor (1946) fue el primero en darse cuenta de que el principio de incertidumbre aplica para señales. Formalmente su teorema:

Para una señal con energía finita 
$$
E = \int |s(t)|^2 dt
$$
con valor medio 
$$
\langle t \rangle = \frac{1}{E} \int t |s(t)|^2 dt,
$$
y varianza temporal
$$
(\Delta t)^2 = \frac{1}{E} \int (t - \langle t \rangle)^2 |s(t)|^2 dt,
$$
cuya transformada de Fourier $\mathbb{FT}[s(t)] = S(\omega)$ tiene un valor medio en frecuencia
$$
\langle \omega \rangle = \frac{1}{E} \int (\omega - \langle \omega \rangle) |S (\omega)|^2 d \omega
$$
y varianza frecuencial
$$
(\Delta \omega)^2 = \frac{1}{E} \int (\omega - \langle \omega \rangle)^2 |S(\omega)|^2 d\omega
$$

***
Entonces se cumple que

$$
\Delta t \Delta \omega \geq \frac{1}{2},
$$

es decir $\Delta t$ y $\Delta \omega$ no pueden ser arbitrariamente pequeños. Esto se conoce también como límite de Gabor

***


***

### Ejemplo: Transformada de Fourier de una Gaussiana

La función Gaussiana se define como

$$
s(t) = \alpha e^{- \beta t^2}
$$
y su transformada de Fourier es

$$
\begin{align}
S(\omega) &= \alpha \int  e^{- \beta t^2} e^{-j\omega t} dt \nonumber \\
&= \alpha e^{- \frac{\omega^2}{4\beta}}\int e^{-\beta (t + \frac{j\omega}{2\beta})^2 } dt \nonumber \\
&= \alpha e^{- \frac{\omega^2}{4\beta}} \sqrt{\frac{\pi}{\beta}} = \hat \alpha e^{- \hat \beta \omega^2}, \nonumber 
\end{align}
$$
es decir otra gaussiana.

La gaussiana es la unica función que cumple $\Delta t \Delta \omega = \frac{1}{2}$

***

## Definiciones

- Función limitada en el tiempo
$$
s(t) = 0 \quad \forall |t| > T,
$$
para alguna constante $T$

- Función limitada en ancho de banda
$$
S(\omega) = 0 \quad \forall |\omega| > \Omega,
$$
para alguna constante $\Omega$

***

## Enventanado

- En la práctica no trabajamos con señales de duración infinita
- Una señal de duración infinita puede hacerse finita multiplicando por una **ventana** finita
- Por ejemplo
$$
s_T(t) = \cos(\omega_0 t) \text{rect}(t/T),
$$
donde 
$$
\text{rect}(x) = \begin{cases} 1 & |x| \leq 1 \\ 0 & |x| > 0 \end{cases}
$$
es una ventana rectangular, *i.e.* pulso cuadrado

¿Cúal es la transformada de Fourier de $s_T(t)$?

$$
\begin{align}
S_T(\omega) &= \int s_T(t) e^{-j\omega t} dt \nonumber \\
&= \int_{-T}^T \cos(\omega_0 t) \cos(\omega t) dt \nonumber \\
&= T \text{sinc}((\omega - \omega_0)T) +  T \text{sinc}((\omega + \omega_0)T) \nonumber 
\end{align}
$$


Donde usamos que 
$$
2 \cos(\alpha)\cos(\beta) = \cos(\alpha - \beta) +  \cos(\alpha + \beta)
$$

Que también se resuelve usando la propiedad de modulación
$$
\begin{align}
S_T(\omega) &= T \text{sinc}((\omega - \omega_0)T) +  T \text{sinc}((\omega + \omega_0)T) \nonumber \\
&=  T \text{sinc}(\omega T) * \left[ \delta(\omega - \omega_0) + \delta(\omega + \omega_0) \right] \nonumber \\
&= \frac{1}{2\pi} \mathbb{FT}[\text{rect}(t/T)] * \mathbb{FT}[\cos(\omega_0 t)] \nonumber
\end{align}
$$

- Es decir que multiplicar por una ventana rectangular en el tiempo es equivalente a convolucionar con un sinc en frecuencia
- ¿Qué repercusión tiene esto en el espectro?

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(6, 4))
f = np.linspace(-3, 3, num=500)
def update(T, window):
    t = np.arange(-T, T, step=1e-2); 
    tf = np.cos(2.0*np.pi*t[:, np.newaxis]*f[:, np.newaxis].T);
    if window == "rect":
        w = np.ones_like(t)
    elif window == "gaussian":
        w = np.exp(-0.5*t**2/(0.333*T)**2)
    s = w*np.cos(2.0*np.pi*t); 
    S = np.average(s[:, np.newaxis]*tf, axis=0)
    ax[0].cla(); ax[1].cla(); ax[0].set_title(r'$s_T(t) = rect(t/T)cos(2\pi t)$')
    ax[0].plot(t, s); ax[1].plot(f, S); ax[1].set_title(r'$\Re[S_T(f)]$')
interact(update, T=SelectionSlider_nice(options=[0.5, 1, 2, 4, 8, 16], value=8),
         window=SelectionSlider_nice(options=["rect", "gaussian"]));

## Fuga espectral

La fuga espectral (spectral leak):
- En una distribución de la energía de un cierto componente espectral hacia frecuencias vecinas
- Aparece como lobulos laterales en torno a los componentes espectrales
- Sólo aparece si la señal es no periódica dentro del rango de observación 
- Está asociado a las discontinuidades provocadas por los bordes de la señal
    - Una discontinuidad fuerte puede ocupar mucho espacio en frecuencia ¿Por qué?

***
Multiplicando la señal por una ventana apropiada podemos reducir este efecto a un rango más acotado de frecuencia
***

### Ventana de Hann

Se define como
$$
w[n] = 0.5 - 0.5 \cos \left( \frac{2\pi n}{N-1} \right), \quad n=0, 1, \ldots, N-1,
$$
y tiene bordes que tienden a cero

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(6, 4))
def update(f0, window, log_scale=False):
    n = np.arange(0, 100);
    if window == "rect":
        w = np.ones_like(n)    
    elif window == "hamming":
        w = 0.5 - 0.5*np.cos(2*np.pi*n/(len(n)))
    s = w*np.cos(2.0*np.pi*f0*n); f = fftpack.fftshift(fftpack.fftfreq(n=len(s), d=1))
    S = fftpack.fftshift(np.absolute(fftpack.fft(s)))
    ax[0].cla(); ax[1].cla(); ax[0].set_title(r'$w[n]cos(2\pi f_0 n)$')
    ax[0].plot(n, s); ax[1].plot(f, S); ax[1].set_title(r'$|S_T(f)|$')
    if log_scale:
        ax[1].set_yscale('log')
interact(update, f0=SelectionSlider_nice(options=[0.05, 0.05214178, 0.055414684, 0.2, 0.241245], value=0.05),
         window=SelectionSlider_nice(options=["rect", "hamming"]));

## Toma 2: Efectos de la frecuencia de muestreo y de la ventana de observación en el espectro

1. **Fila superior, linea azul**: señal con frecuencia de muestreo 50 Hz y duración 10 segundos
1. **Superior izquierda (cruces rojas)**: señal con un décimo de la frecuencia de muestreo pero igual largo temporal
1. **Superior derecha (cruces rojas)**: señal con igual frecuencia de muestreo pero con un décimo del largo temporal



> La frecuencia de muestreo influye en la frecuencia máxima que podemos estudiar (frecuencia de Nyquist)

> El largo temporal o ventana de observación influye en la resolución frecuencial del espectro 


In [ ]:
x_plot = np.cos(2.0*np.pi*f0*t_plot) + 0.4*np.cos(2.0*np.pi*2*f0*t_plot +np.pi/4) + 0.1*np.random.randn(1000)

t, x = t_plot[::10], x_plot[::10]
X = fftpack.fft(x)[:len(x)//2]
freq = fftpack.fftfreq(n=len(x), d=t[1]-t[0])[:len(x)//2]

fig, ax = plt.subplots(2, 2, figsize=(7, 4), tight_layout=True)
ax[0, 0].plot(t_plot,x_plot)
ax[0, 0].scatter(t, x, marker='x', c='r')
ax[1, 0].plot(freq, np.absolute(X))

t, x = t_plot[:100], x_plot[:100]
X = fftpack.fft(x)[:len(x)//2]
freq = fftpack.fftfreq(n=len(x), d=t[1]-t[0])[:len(x)//2]
ax[0, 1].plot(t_plot,x_plot)
ax[0, 1].scatter(t, x, marker='x', c='r')
ax[1, 1].plot(freq, np.absolute(X));

## Toma 2: Incertidumbre frecuencia-tiempo

Mientras más pequeño es la "separación en frecuencia" de dos señales, más tiempo debemos observarlas para poder diferenciarlas

In [ ]:
t, dt = np.linspace(0.0, 5.0, num=1000, retstep=True)

fig, ax = plt.subplots(4, figsize=(8, 6), tight_layout=True)

for k in range(4):
    ax[3-k].plot(t, np.cos(2.0*np.pi*2.1*t))
    ax[3-k].plot(t, np.cos(2.0*np.pi*2.1*(1+0.01+0.01*k)*t))
    ax[3-k].set_title('Error relativo: %0.2f' %(0.01+0.01*k))